In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Charger les données
df = pd.read_csv('Dataset.csv')

In [36]:
display(df.SepsisLabel.value_counts())

SepsisLabel
0    1524294
1      27916
Name: count, dtype: int64

In [37]:
# Préparation des données
def preprocess_data(df, scaler=None):
    
    # Transformation de la colonne catégorielle Gender
    df = pd.get_dummies(df, columns=['Gender'], drop_first=True)
    
    # Normalisation des colonnes d'intérêt
    columns_normalized = ['MAP', 'BUN', 'Creatinine', 'Glucose', 'WBC', 'Platelets']
    for col in columns_normalized:
        df[col] = np.log(df[col] + 1)
    
    # Isoler les labels avant la normalisation
    labels = df['SepsisLabel']

    # Supprimer les colonnes non nécessaires
    df = df.drop(['Patient_ID', 'Unnamed: 0', 'SepsisLabel'], axis=1)
    
    # Normalisation des données
    if scaler is None:
        scaler = StandardScaler()
        df[df.columns] = scaler.fit_transform(df[df.columns])
    else:
        df[df.columns] = scaler.transform(df[df.columns])
        
    # Réintégrer les labels après la normalisation
    df['SepsisLabel'] = labels
    
    return df, scaler

# Vérification et correction des labels SepsisLabel pour s'assurer qu'ils ne contiennent que les valeurs 0 et 1
df['SepsisLabel'] = df['SepsisLabel'].apply(lambda x: 1 if x > 0 else 0)

# Division des données en ensembles d'entraînement et de test
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Appliquer la fonction preprocess_data sur les ensembles d'entraînement et de test
df_train, scaler = preprocess_data(df_train)
df_test, _ = preprocess_data(df_test, scaler)

# Extraction des caractéristiques et des labels
X_train = df_train.drop('SepsisLabel', axis=1).values
y_train = df_train['SepsisLabel'].values

X_test = df_test.drop('SepsisLabel', axis=1).values
y_test = df_test['SepsisLabel'].values

# Reshape des données pour LSTM (échantillons, séquences, caractéristiques)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Encodage one-hot des labels
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)



In [38]:
print(X_train.shape, X_test.shape)

(1241768, 1, 41) (310442, 1, 41)


In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Définition du modèle LSTM
model = Sequential()

# Ajout d'une première couche LSTM avec 64 neurones et input_shape adapté à vos données
model.add(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

# Ajout d'une deuxième couche LSTM avec 32 neurones
model.add(LSTM(units=32))
model.add(Dropout(0.2))

# Couche Dense pour la classification binaire avec activation softmax
model.add(Dense(units=2, activation='softmax'))

# Compilation du modèle avec optimiseur Adam et fonction de perte categorical_crossentropy
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Affichage de l'architecture du modèle
model.summary()


C:\Users\AdminPC\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 1, 64)               │          27,136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 39,618 (154.76 KB)

 Trainable params: 39,618 (154.76 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
# Entraînement du modèle avec X_train et y_train
history = model.fit(X_train, y_train, epochs=20, batch_size=128, validation_data=(X_test, y_test), verbose=1)

# Évaluation du modèle sur les données de test
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Epoch 1/20
9702/9702 ━━━━━━━━━━━━━━━━━━━━ 45s 4ms/step - accuracy: 0.9818 - loss: nan - val_accuracy: 0.9822 - val_loss: nan
Epoch 2/20
9702/9702 ━━━━━━━━━━━━━━━━━━━━ 53s 5ms/step - accuracy: 0.9820 - loss: nan - val_accuracy: 0.9822 - val_loss: nan
Epoch 3/20
9702/9702 ━━━━━━━━━━━━━━━━━━━━ 52s 5ms/step - accuracy: 0.9819 - loss: nan - val_accuracy: 0.9822 - val_loss: nan
Epoch 4/20
9702/9702 ━━━━━━━━━━━━━━━━━━━━ 53s 5ms/step - accuracy: 0.9818 - loss: nan - val_accuracy: 0.9822 - val_loss: nan
Epoch 5/20
9702/9702 ━━━━━━━━━━━━━━━━━━━━ 52s 5ms/step - accuracy: 0.9820 - loss: nan - val_accuracy: 0.9822 - val_loss: nan
Epoch 6/20
9702/9702 ━━━━━━━━━━━━━━━━━━━━ 53s 5ms/step - accuracy: 0.9822 - loss: nan - val_accuracy: 0.9822 - val_loss: nan
Epoch 7/20
9702/9702 ━━━━━━━━━━━━━━━━━━━━ 53s 5ms/step - accuracy: 0.9820 - loss: nan - val_accuracy: 0.9822 - val_loss: nan
Epoch 8/20
9702/9702 ━━━━━━━━━━━━━━━━━━━━ 52s 5ms/step - accuracy: 0.9818 - loss: nan - val_accuracy: 0.9822 - val_loss: nan


In [46]:
# Évaluation du modèle
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, y_pred_classes))


9702/9702 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99    304903
           1       0.00      0.00      0.00      5539

    accuracy                           0.98    310442
   macro avg       0.49      0.50      0.50    310442
weighted avg       0.96      0.98      0.97    310442



C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
